## Library

In [ ]:
import pickle

import pandas as pd
import os 

import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras import layers, models #Tensorflow에 있는 Keras 함수들 호출하기
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt


## Train & Test set

In [ ]:
train_img = np.load('./train_resize_224.npy')
train_hair_img = np.load('./np_train_img_type1.npy')
train_age = np.load('./train_age.npy')
train_gender = np.load('./train_gender.npy')
train_hair = np.load('./train_hair.npy')
train_hair_label = np.load('./np_hairstyle_label_type1.npy')

In [ ]:
test_img = np.load('./test_resize_224.npy')
test_age = np.load('./test_age.npy')
test_gender = np.load('./test_gender.npy')
test_hair = np.load('./test_hair.npy')

In [ ]:
train_input = train_img
train_target = train_hair
test_input = test_img
test_target = test_hair

In [ ]:
train_input.shape, train_target.shape, test_input.shape, test_target.shape

((8071, 224, 224, 3), (8071,), (1010, 224, 224, 3), (1010,))

In [ ]:
n_class = np.unique(train_target).shape[0]
n_class

5

In [ ]:
# train_input, train_target = train_input / 255.0, train_target / 255.0

## Zero-centering

In [ ]:
def zero_mean(image):
    # zero-centering
    return np.mean(image, axis=0)

train_input = train_input.astype('float32')
test_input = test_input.astype('float32')

zero_mean_img = zero_mean(trian_input)
zero_mean_img_test = zero_mean(test_input)

train_input -= zero_mean_img
train_input -= zero_mean_img

In [ ]:
zero_mean_img.shape
zero_mean_img_t.shape

(224, 224, 3)

## CNN

In [ ]:
#모델 구축
model = models.Sequential()
## filtering layer
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))  
model.add(layers.MaxPooling2D((2, 2)))  
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))  
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

## classification layer
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
#모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Early stopping

In [ ]:
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', #-> 성능 판단 지표  
    min_delta=0, #-> 개선이라 판단할 최소값    
    patience=0, #-> 개선되지않을때 몇 epoch기다렸다 stop할것인지   
    restore_best_weights=False #-> 최적weight값으로 복원할지  
    ) 

Reduce Learning rate

In [ ]:
tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.1, # new_lr = lr * factor.
    patience=10,
    min_delta=0.0001, # threshold for measuring the new optimum, to only focus on significant changes.  
    cooldown=0, # number of epochs to wait before resuming normal operation after lr has been reduced.  
    min_lr=0
)

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0)


In [ ]:
#모델 학습
with tf.device('CPU'):
    model.fit(train_input, train_target, epochs = 30, validation_split=0.2, batch_size= 125, callbacks=[early, callback])

    # model.fit(train_input, train_target, epochs = 30, validation_split=0.2, callbacks=[callback])



Epoch 1/30
52/52 [==============================] - 380s 7s/step - loss: 25.8630 - accuracy: 0.5240 - val_loss: 1.0397 - val_accuracy: 0.5251
Epoch 2/30
52/52 [==============================] - 437s 8s/step - loss: 0.8713 - accuracy: 0.6193 - val_loss: 0.9601 - val_accuracy: 0.5771
Epoch 3/30
22/52 [===========>..................] - ETA: 3:52 - loss: 0.8221 - accuracy: 0.6415

In [ ]:
#모델 검증
with tf.device('CPU'):
    test_loss, test_acc = model.evaluate(test_input, test_target, verbose=2)
    print(test_acc)